In [1]:
import os
import cv2
import torch
import minetorch

import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as albu
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp


from tqdm import tqdm
from models.mvssnet import get_mvss
from albumentations.pytorch import ToTensorV2
from albumentations import (Normalize, Compose)
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

In [2]:
test_path = './data/ze.csv'
model_name = 'timm-efficientnet-b0'
model_paths = ['./ckpt/mvssnet_casia.pt']
# model_paths = ['./efficientnet/fold-0/models/latest.pth.tar']
save_path = './result'

width = 512
height = 512
batchsize = 1

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
class FakeDataset:
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = row['image']
        image = cv2.imread(path)
        img_ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        shape = img_ori.shape
        augmented = self.transforms(image=img_ori)
        img = augmented['image']

        return img.float(), path.split('/')[-1], shape

    def __len__(self):
        return len(self.df)

def make_transforms(phase,mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), tta=False):
    list_transforms = []
    if tta:
        list_transforms.extend(
            [
                albu.HorizontalFlip(p=1),
                
            ]
        )
    list_transforms.extend(
        [
            albu.Resize(width, height),
            albu.UnsharpMask(p=1),
            ToTensorV2(),
        ]
    )

    list_trfms = Compose(list_transforms)
    return list_trfms

In [4]:
df = pd.read_csv(test_path)
    

testset = FakeDataset(df, make_transforms('val'))
testset_tta = FakeDataset(df, make_transforms('val', tta=True))

test_loader = DataLoader(
    testset,
    batch_size=batchsize,
    num_workers=8,
    pin_memory=True
)

test_loader_tta = DataLoader(
    testset_tta,
    batch_size=batchsize,
    num_workers=8,
    pin_memory=True
)

In [5]:
models = []

for model_path in model_paths:
    '''model = smp.UnetPlusPlus(
        model_name,
        classes=1,
        encoder_weights=None,
        activation=None,
    ).cuda()'''
    model = get_mvss(backbone='resnet50',
                 pretrained_base=True,
                 nclass=1,
                 sobel=True,
                 constrain=True,
                 n_input=3,
    ).cuda()
    stuff = torch.load(model_path)
    model.load_state_dict(stuff)
    model.eval();
    models.append(model)

load pretrain success
----------use sobel-------------
----------use constrain-------------
load pretrain success


In [13]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

file_name = 'images'
if not os.path.isdir(os.path.join(save_path, file_name)):
    os.makedirs(os.path.join(save_path, file_name))

for (image, fn, shape), (image_tta, _, _) in tqdm(zip(test_loader, test_loader_tta), total=len(testset)):
    mask1 = 0
    for model in models:
        masks1 = model(image.cuda())
        masks_tta1 = model(image_tta.cuda())
        mask1 += albu.Resize(shape[0].item(), shape[1].item())(image=masks1[0].permute(1,2,0).detach().cpu().numpy())['image']
        mask1 += albu.Resize(shape[0].item(), shape[1].item())(image=np.flip(masks_tta1[0].permute(1,2,0).detach().cpu().numpy(), axis=1))['image']
        mask1 /= 2
    mask1 /= len(models)
    save_seg_path = os.path.join(save_path, file_name, fn[0].split('.')[0] + '.png')
    fake_mask = ((sigmoid(mask1) > 0.5)*255.).astype(np.uint8)
    cv2.imwrite(save_seg_path, fake_mask.astype(np.uint8))

(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)(1080, 1920, 3)







(1080, 1920, 3)
(1080, 1920, 3)


  0%|                                                    | 0/10 [00:00<?, ?it/s]

(1080, 1920, 3)
(1080, 1920, 3)(1080, 1920, 3)
(1080, 1920, 3)(1080, 1920, 3)
(1080, 1920, 3)

(1080, 1920, 3)

(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)


  0%|                                                    | 0/10 [00:00<?, ?it/s]


RuntimeError: number of dims don't match in permute

In [9]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

file_name = 'ze'
if not os.path.isdir(os.path.join(save_path, file_name)):
    os.makedirs(os.path.join(save_path, file_name))

for (image, fn, shape), (image_tta, _, _) in tqdm(zip(test_loader, test_loader_tta), total=len(testset)):
    mask1 = 0
    for model in models:
        _, masks1 = model(image.cuda())
        _, masks_tta1 = model(image_tta.cuda())
        print(masks1.shape)
        masks1 = masks1.squeeze(1)
        masks_tta1 =  masks_tta1.squeeze(1)
        mask1 += albu.Resize(shape[0].item(), shape[1].item())(image=masks1.permute(1,2,0).detach().cpu().numpy())['image']
        mask1 += albu.Resize(shape[0].item(), shape[1].item())(image=np.flip(masks_tta1.permute(1,2,0).detach().cpu().numpy(), axis=1))['image']
        mask1 /= 2
    mask1 /= len(models)
    save_seg_path = os.path.join(save_path, file_name, fn[0].split('.')[0] + '.png')
    fake_mask = ((sigmoid(mask1) > 0.5)*255.).astype(np.uint8)
    cv2.imwrite(save_seg_path, fake_mask.astype(np.uint8))

  0%|                                                    | 0/10 [00:00<?, ?it/s]/opt/anaconda3/envs/mvss/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
 10%|████▍                                       | 1/10 [00:00<00:02,  3.18it/s]

torch.Size([1, 1, 512, 512])


 20%|████████▊                                   | 2/10 [00:00<00:02,  3.67it/s]

torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])


 60%|██████████████████████████▍                 | 6/10 [00:00<00:00,  5.49it/s]

torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])


 80%|███████████████████████████████████▏        | 8/10 [00:01<00:00,  6.46it/s]

torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])
torch.Size([1, 1, 512, 512])


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.39it/s]
